# Data Cleaning

## Introduction

Varibles such as stress and motivation can have both positive and negative affective states on performers.  Subsequently players are asked to submit a tweet-length assessment of their physical, mental and emotiona status in their own words.  What I'm looking for are signs of burnout - where sentiment might change from positive or negative or from fact based to opinion based, and vice versa.   It's important not to assume positive sentiment correlates with burnout. what we're looking for are significant changes in personality as expressed in sentiment.  Burnout is associated with the CHANGE not the affective state of the change; 

Secondly, if there is a change, it can give us a better understaning around the persoal motivational constructs and goal orientation of the athlete.  

1. **Get 'Tweet' from database
2. **Clean the text
3. **Organizing the data
4. **Render as JSON in app for real-time updates. 

The output of this notebook will render two JSON routes

1. **tweet** - a collection of text
2. **Document-Term Matrix** - word counts in matrix format to run word count on

In [28]:
#pull the data
import pandas as pd
data = pd.read_csv('derby_scored.csv')
data.head()

,timestamp,org,sport,pos,player,jump_score,sleep,nutrition,fatigue,motivation,stress,RPE,plus,value,tweet
0,1/17/17,AZRD,Roller Derby,pivot,Havoc,1.99,8,7,9,10,8,9,-1,1,"""To affect the quality of the day, that is the highest of arts."""
1,1/17/17,AZRD,Roller Derby,blocker,Kootz,1.92,5,7,6,6,8,7,-3,1,"""A man is what he thinks about all day long."""
2,1/17/17,AZRD,Roller Derby,blocker,Monroe,1.86,8,5,5,7,10,1,4,1,"""If you want a thing done well, do it yourself."""
3,1/17/17,AZRD,Roller Derby,blocker,Calaca,1.69,6,7,8,10,7,5,3,1,"""The backbone of surprise is fusing speed with secrecy."""
4,5/23/17,AZRD,Roller Derby,blocker,Ginger,1.11,7,8,7,7,8,8,-1,1,"""Each day is the scholar of yesterday."""


In [29]:
#Get Essential data only
import pandas as pd
#pd.set_option('max_colwidth',150)

#data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df = data[['timestamp','player','tweet']]
data_df = data_df.sort_index()
data_df

,timestamp,player,tweet
0,1/17/17,Havoc,"""To affect the quality of the day, that is the highest of arts."""
1,1/17/17,Kootz,"""A man is what he thinks about all day long."""
2,1/17/17,Monroe,"""If you want a thing done well, do it yourself."""
3,1/17/17,Calaca,"""The backbone of surprise is fusing speed with secrecy."""
4,5/23/17,Ginger,"""Each day is the scholar of yesterday."""
...,...,...,...
346,6/18/19,Chaos,"""When one burns one's bridges, what a very nice fire it makes."""
347,6/18/19,FridaFrida,"""Imagination and fiction make up more than three quarters of our real life."""
348,6/19/19,FridaFrida,"""The pen is the tongue of the mind."""
349,6/19/19,FridaFrida,"""We have, I fear, confused power with greatness."""


In [66]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [67]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_df.tweet.apply(round1))
data_clean

,tweet
0,to affect the quality of the day that is the highest of arts
1,a man is what he thinks about all day long
2,if you want a thing done well do it yourself
3,the backbone of surprise is fusing speed with secrecy
4,each day is the scholar of yesterday
...,...
346,when one burns ones bridges what a very nice fire it makes
347,imagination and fiction make up more than three quarters of our real life
348,the pen is the tongue of the mind
349,we have i fear confused power with greatness


In [68]:
# data_clean["player"]= data_df["player"]
# total_tweets = data_clean.groupby("player")
# total_tweets = pd.DataFrame(total_tweets)

# total_tweets.rename(columns = {0:'player', 1:'tweet'}, inplace = True)
# total_tweets.to_csv("total_tweets.csv")
# total_tweets

In [69]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [70]:
# clean_tweet = pd.DataFrame(total_tweets.tweet.apply(round1))
# clean_tweet = pd.DataFrame(total_tweets.tweet.apply(round2))
# clean_tweet = pd.DataFrame(clean_tweet)
# clean_tweet

In [71]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean.tweet.apply(round2))
data_clean[["player", "timestamp"]]= data_df[["player", "timestamp"]]
data_clean

,tweet,player,timestamp
0,to affect the quality of the day that is the highest of arts,Havoc,1/17/17
1,a man is what he thinks about all day long,Kootz,1/17/17
2,if you want a thing done well do it yourself,Monroe,1/17/17
3,the backbone of surprise is fusing speed with secrecy,Calaca,1/17/17
4,each day is the scholar of yesterday,Ginger,5/23/17
...,...,...,...
346,when one burns ones bridges what a very nice fire it makes,Chaos,6/18/19
347,imagination and fiction make up more than three quarters of our real life,FridaFrida,6/18/19
348,the pen is the tongue of the mind,FridaFrida,6/19/19
349,we have i fear confused power with greatness,FridaFrida,6/19/19


## Organizing The Data

The output of this notebook will be in two standard text formats:
1. **tweets - **a collection of text
2. **Document-Term Matrix - **word counts in matrix format

In [72]:
#pickle it for later use
data_clean.to_pickle("tweet.pkl")
data_clean.to_csv("data_clean.csv")

### Document-Term Matrix

For the next analysis the text must be tokenizedinto smaller pieces. Breaking down text into words is a common tokenization technique. We can use scikit-learn's CountVectorizer. 

CountVectorizer can also remove stop words. Stop words are common words that add no additional meaning to text such as 'a', 'the', etc.

In [73]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.tweet)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

,ability,able,absence,absent,abstain,abstains,abundance,accept,acceptable,accepting,...,year,years,yes,yesterday,youd,young,youre,youth,youve,zeal
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
347,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
348,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
349,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
#pickle it for later use
data_dtm.to_pickle("dtm.pkl")
data_dtm.to_csv("dtm.csv")

In [76]:
import pickle
# Let's also pickle the cleaned data (before we put it in document-term matrix format) and the CountVectorizer object
data_clean.to_pickle('data_clean.pkl')
data_clean.to_csv('data_clean.csv')
pickle.dump(cv, open("cv.pkl", "wb"))